In [2]:
import pandas as pd
import numpy as np

In [3]:
file_path = 'rates.csv'

The `data` dataframe will only be used when initially loading the CSV file

In [4]:
data = pd.read_csv(file_path, header=None)

In [5]:
data

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
0,Date,1Y01Y,Date,1Y02Y,Date,1Y03Y,Date,1Y05Y,Date,1Y10Y,...,Date,FY03Y,Date,FY05Y,Date,FY10Y,Date,FY20Y,Date,FY30Y
1,09-Nov-11,1.31,24-Nov-11,0.972,24-Nov-11,1.207,24-Nov-11,1.701,24-Nov-11,2.336,...,24-Nov-11,1.04,24-Nov-11,1.52,24-Nov-11,2.22,24-Nov-11,2.56,24-Nov-11,2.62
2,10-Nov-11,0.72,25-Nov-11,0.996,25-Nov-11,1.25,25-Nov-11,1.76,25-Nov-11,2.408,...,25-Nov-11,1.07,25-Nov-11,1.57,25-Nov-11,2.29,25-Nov-11,2.64,25-Nov-11,2.69
3,11-Nov-11,0.78,28-Nov-11,0.989,28-Nov-11,1.243,28-Nov-11,1.76,28-Nov-11,2.421,...,28-Nov-11,1.06,28-Nov-11,1.56,28-Nov-11,2.3,28-Nov-11,2.64,28-Nov-11,2.69
4,14-Nov-11,0.75,29-Nov-11,0.965,29-Nov-11,1.215,29-Nov-11,1.745,29-Nov-11,2.429,...,29-Nov-11,1.03,29-Nov-11,1.54,29-Nov-11,2.3,29-Nov-11,2.66,29-Nov-11,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,30-Jun-23,4.502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3039,03-Jul-23,4.586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3040,04-Jul-23,4.586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3041,05-Jul-23,4.568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The array `fwds` will contain the labels for each of the series of fwd rates, any `nan`s at the end of the CSV file are filtered out of it

In [6]:
fwds = data.iloc[0, :].unique()
fwds = fwds[np.array([isinstance(item, str) for item in fwds])][1:]

The working dataframe will be called `df` and starts as an empty dataframe.  

We iterate through the fwds array getting each `label` and use an index `i` to move across the `data` dataframe.
At each iteration;
1. create a new dataframe _imaginatively_ called `s` which contains each series of dates and forwards
2. we drop any `nan` values from the dates
3. we convert the dates from strings to datetime objects
4. make the date column the index of `s`
5. rename the column of fwds with `label`
6. concatenate to the `df` dataframe

In [7]:
df = pd.DataFrame()
i = 0
for label in fwds:
    s = data.iloc[1:, i:i+2]
    s = s.dropna(subset=[i])
    s[i] = pd.to_datetime(s[i], format=('%d-%b-%y'))
    s = s.set_index(s.columns[0])
    s = s.rename(columns={i+1: label})
    s[label] = s[label].replace('00/01/1900', np.nan)
    s[label] = s[label].astype(float)
    df = pd.concat([df,s],axis=1)
    i += 2

In [8]:
df = df.sort_index(ascending=True)
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [9]:
df.dtypes

1Y01Y    float64
1Y02Y    float64
1Y03Y    float64
1Y05Y    float64
1Y10Y    float64
          ...   
FY03Y    float64
FY05Y    float64
FY10Y    float64
FY20Y    float64
FY30Y    float64
Length: 91, dtype: object